# Title

---
embed-resources: true
---

## Introduction

## Methods

In [ ]:
# imports

### Data

In [ ]:
# load data

In [ ]:
# summary statistics

In [ ]:
# exploratory visualization

### Models

In [ ]:
# process data for ML

In [ ]:
# train models

## Results

In [ ]:
# report model metrics

In [ ]:
# summary figure

In [ ]:
# serialize model

## Discussion